In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataset import Dataset
import numpy as np

In [3]:
import argparse
parser = argparse.ArgumentParser(description='Run Diagnosis experiments')
parser.add_argument("--data_dir", type=str, required=True)
parser.add_argument('--display', dest='display', action='store_true')
parser.add_argument("--output_dir", type=str)
parser.add_argument("--mock", dest='mock', action='store_true')

args = parser.parse_args(['--data_dir=.', '--output_dir=outputs/', '--display'])

In [4]:
data = Dataset(name='imdb', dirname='preprocess/IMDB/')

INFO - 2019-03-11 12:12:41,733 - Reading Structured data ...
INFO - 2019-03-11 12:12:41,744 - Reading Notes ...
INFO - 2019-03-11 12:12:42,592 - Stratifying ...
WARNING - 2019-03-11 12:12:42,605 - No records for dev split


In [5]:
labellist = ['sentiment']
data.generate_labels(labellist, len(labellist), 'binary')
data.save_on_metric = 'roc_auc'
data.metrics_type = 'classifier'

In [6]:
from dataloaders import readmission_dataset
data = readmission_dataset(args)

INFO - 2019-03-11 12:12:44,085 - Reading Structured data ...
INFO - 2019-03-11 12:12:44,243 - Reading Notes ...
INFO - 2019-03-11 12:13:02,674 - Stratifying ...


In [7]:
from PatientVec.Experiments.hyperparam_exps import *

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [8]:
train_data, dev_data = get_basic_data(data, structured=True, truncate=90)

Pos Percentage [0.19738451]


INFO - 2019-03-11 12:13:13,951 - Maximum Sentence Length 721825.000000 , 90 percentile length 18689.000000 ... 
INFO - 2019-03-11 12:13:39,410 - Truncated all ...


Pos Percentage [0.19429882]


INFO - 2019-03-11 12:13:42,930 - Maximum Sentence Length 496156.000000 , 90 percentile length 18936.000000 ... 
INFO - 2019-03-11 12:13:47,277 - Truncated all ...


In [43]:
config = {'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 'type' : data.metrics_type, 'norm' : 'l2'}
lr = LR(config)
lr.train(train_data)
lr.evaluate(dev_data, save_results=True)


Normalising using l2
Fit BOW Classifier ...



Normalising using l2
BOW
{'accuracy': 0.7051114023591087, 'roc_auc': 0.7450634475093615, 'pr_auc': 0.44835366773317537}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.798,0.457,0.705,0.627,0.731
precision,0.892,0.356,0.705,0.624,0.788
recall,0.721,0.637,0.705,0.679,0.705
support,7377.000,1779.000,9156.000,9156.000,9156.000


In [9]:
data.generate_bowder(train_data, stop_words=True, norm='l2')
train_data.X = data.get_vec_encoding(train_data, _type='bow')
dev_data.X = data.get_vec_encoding(dev_data, _type='bow')


Normalising using l2



Normalising using l2


In [23]:
e = vector_configs[0]
config = e(data, structured=True, args=args)
config['model']['reg']['weight'] = 1.0 / train_data.X.shape[0]
config['training_config']['groups'][0][1]['lr'] = 0.01
config

{'model': {'type': 'vec_classifier',
  'decoder': {'num_layers': 1,
   'hidden_dims': [1],
   'activations': ['linear'],
   'dropout': 0.2,
   'input_dim': 32101},
  'predictor': {'type': 'binary'},
  'structured': {'use_structured': True, 'structured_dim': 142},
  'reg': {'type': 'l1', 'weight': 3.3967391304347826e-05}},
 'training_config': {'type': 'Adam',
  'groups': [('.*', {'lr': 0.01, 'weight_decay': 0.0})],
  'common': {'bsize': 128, 'class_weight': True}},
 'exp_config': {'exp_name': 'Readmission/baselines/Decoder()+Structured'}}

In [ ]:
trainer = Trainer(VectorCT, config, _type=data.metrics_type, display_metrics=args.display)
trainer.train(train_data, dev_data, save_on_metric=data.save_on_metric)

INFO - 2019-03-11 13:24:50,075 - instantiating class <class 'PatientVec.models.Model.Model'> from params {'type': 'vec_classifier', 'decoder': {'num_layers': 1, 'hidden_dims': [1], 'activations': ['linear'], 'dropout': 0.2, 'input_dim': 32101}, 'predictor': {'type': 'binary'}, 'structured': {'use_structured': True, 'structured_dim': 142}, 'reg': {'type': 'l1', 'weight': 3.3967391304347826e-05}} and extras {}
INFO - 2019-03-11 13:24:50,075 - type = vec_classifier
INFO - 2019-03-11 13:24:50,076 - decoder.input_dim = 32243
INFO - 2019-03-11 13:24:50,076 - decoder.num_layers = 1
INFO - 2019-03-11 13:24:50,077 - decoder.hidden_dims = [1]
INFO - 2019-03-11 13:24:50,077 - decoder.activations = ['linear']
INFO - 2019-03-11 13:24:50,078 - decoder.dropout = 0.2
INFO - 2019-03-11 13:24:50,080 - instantiating class <class 'PatientVec.models.modules.Predictor.Predictor'> from params {'type': 'binary'} and extras {}
INFO - 2019-03-11 13:24:50,081 - predictor.type = binary
INFO - 2019-03-11 13:24:50,

{'accuracy': 0.7100262123197904, 'roc_auc': 0.7417830040545783, 'pr_auc': 0.44136609535783433}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.802,0.457,0.71,0.630,0.735
precision,0.891,0.359,0.71,0.625,0.788
recall,0.730,0.629,0.71,0.679,0.710
support,7377.000,1779.000,9156.00,9156.000,9156.000


Model Saved on  roc_auc 0.7417830040545783


{'accuracy': 0.6027741371778069, 'roc_auc': 0.7232763851427987, 'pr_auc': 0.39138930469570504}


...

,0,1,micro avg,macro avg,weighted avg
f1-score,0.837,0.472,0.751,0.655,0.766
precision,0.885,0.402,0.751,0.643,0.791
recall,0.795,0.571,0.751,0.683,0.751
support,7377.000,1779.000,9156.000,9156.000,9156.000


Model not saved on  roc_auc 0.7551252571400879


{'accuracy': 0.7621231979030144, 'roc_auc': 0.7512013586429968, 'pr_auc': 0.4567734172095548}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.847,0.469,0.762,0.658,0.773
precision,0.880,0.414,0.762,0.647,0.790
recall,0.816,0.541,0.762,0.678,0.762
support,7377.000,1779.000,9156.000,9156.000,9156.000


Model not saved on  roc_auc 0.7512013586429968


In [51]:
evaluator = Evaluator(VectorCT, trainer.model.dirname, _type=data.metrics_type, display_metrics=args.display)
_ = evaluator.evaluate(dev_data, save_results=True)

INFO - 2019-03-11 11:24:24,732 - instantiating class <class 'PatientVec.models.Model.Model'> from params {'type': 'vec_classifier', 'decoder': {'num_layers': 1, 'hidden_dims': [1], 'activations': ['linear'], 'input_dim': 32101}, 'predictor': {'type': 'binary'}, 'structured': {'use_structured': False}, 'reg': {'type': 'l1', 'weight': 3.3967391304347826e-05}} and extras {}
INFO - 2019-03-11 11:24:24,733 - type = vec_classifier
INFO - 2019-03-11 11:24:24,734 - decoder.input_dim = 32101
INFO - 2019-03-11 11:24:24,735 - decoder.num_layers = 1
INFO - 2019-03-11 11:24:24,735 - decoder.hidden_dims = [1]
INFO - 2019-03-11 11:24:24,736 - decoder.activations = ['linear']
INFO - 2019-03-11 11:24:24,737 - decoder.dropout = 0.0
INFO - 2019-03-11 11:24:24,740 - instantiating class <class 'PatientVec.models.modules.Predictor.Predictor'> from params {'type': 'binary'} and extras {}
INFO - 2019-03-11 11:24:24,741 - predictor.type = binary
INFO - 2019-03-11 11:24:24,741 - instantiating class <class 'Pati


{'accuracy': 0.7348186981214504, 'roc_auc': 0.7591904269556039, 'pr_auc': 0.45953859574644496}


,0,1,micro avg,macro avg,weighted avg
f1-score,0.822,0.477,0.735,0.650,0.755
precision,0.893,0.387,0.735,0.640,0.795
recall,0.762,0.622,0.735,0.692,0.735
support,7377.000,1779.000,9156.000,9156.000,9156.000


In [52]:
for n, v in evaluator.model.model.decoder.named_parameters() :
    print(n, v)

_linear_layers.0.weight Parameter containing:
tensor([[ 0.0355, -0.0117,  0.0377,  ..., -0.0144, -0.0052,  0.0840]],
       device='cuda:0', requires_grad=True)
_linear_layers.0.bias Parameter containing:
tensor([-0.0935], device='cuda:0', requires_grad=True)


In [58]:
weight = evaluator.model.model.decoder._linear_layers[0].weight[0].cpu().data.numpy()

In [64]:
idx = np.argsort(weight)[:10]
idx, weight[idx]

(array([14013, 26702, 21342, 12435, 28088, 30974,  5651,  4740,  6966,
        25746]),
 array([-3.524076 , -2.312232 , -2.2537048, -2.1441844, -2.109638 ,
        -2.0832996, -2.069614 , -2.060762 , -1.9229028, -1.9027059],
       dtype=float32))

In [65]:
np.array(data.bowder.words_to_keep)[idx]

array(['overdose', 'laceration', 'incisional', 'ace', 'lightened',
       'ambulating', 'mvc', 'postoperative', 'oob', 'understanding'],
      dtype='<U37')

In [ ]:
train_data = data.filter_data_length(data.get_data('train', structured=False), 95)
test_data = data.filter_data_length(data.get_data('test', structured=False), 95)

In [ ]:
vc = [0] * data.vocab.vocab_size
for i, v in data.vocab.idx2word.items() :
    vc[i] = v

In [ ]:
from PatientVec.Experiments.modifiable_config_exp import attention_configs
avg_attn_config = attention_configs[0](data, structured=False, args=args)

In [ ]:
avg_attn_config['model']['embedder']['type'] = 'elmo_embedder'
del avg_attn_config['model']['embedder']['embedding_file']
avg_attn_config['model']['embedder']['elmo_options'] = {
    'options_file' : '../../elmo_2x2048_256_2048cnn_1xhighway_options.json',
    'weight_file' : '../../elmo_2x2048_256_2048cnn_1xhighway_weights.hdf5',
    'vocab_to_cache' : vc,
    'scalar_mix_parameters' : [0.3, 0.3, 0.3]
}

In [ ]:
avg_attn_config['training_config']['common']['bsize'] = 64

In [ ]:
from PatientVec.models.Vanilla import ClassificationTrainer as BasicCT
from PatientVec.trainer import Trainer, Evaluator

In [ ]:
import logging.config
logging.config.dictConfig({
    'version': 1,
    'disable_existing_loggers': True
})

In [ ]:
trainer = Trainer(BasicCT, avg_attn_config, _type=data.metrics_type, display_metrics=args.display)
trainer.train(train_data, test_data, save_on_metric=data.save_on_metric)

In [ ]:
outputs = trainer.model.evaluate(test_data.mock(200))

In [ ]:
for n, v in trainer.model.model.named_parameters() :
    print(n, v)